## Web Scraping

We create this file for testing our web scrapping code because this notebook is easy to debug.<br/>
We will explain our code in a comment on the code. <br/> 
reference link: https://weather.com/weather/tenday/l/Boston+MA?canonicalCityId=6320cadd3d539b434b5a45c094becf3edbe8ea88958185a2287a801115c9ae30

- Create request and split the result by line

In [1]:
import requests

r = requests.get('https://weather.com/weather/tenday/l/Boston+MA?canonicalCityId=6320cadd3d539b434b5a45c094becf3edbe8ea88958185a2287a801115c9ae30')
lines = r.text.split('\n')
lines[2:4]

['</icon><div class="twc-table-shadow sticky" className="twc-table-shadow sticky"></div></div><span class="day-detail clearfix">DEC 4</span></div></td><td class="description" headers="description" title="Cloudy skies early, followed by partial clearing. High near 40F. Winds WSW at 5 to 10 mph." data-track-string="ls_24_hour_ls_24_hour_toggle" className="description"><span>AM Clouds/PM Sun</span></td><td class="temp" headers="hi-lo" title="Cloudy skies early, followed by partial clearing. High near 40F. Winds WSW at 5 to 10 mph." data-track-string="ls_24_hour_ls_24_hour_toggle" className="temp"><div><span class="">40<sup>°</sup></span><span class="slash" className="slash"></span><span class="">30<sup>°</sup></span></div></td><td class="precip" headers="precip" title="Cloudy skies early, followed by partial clearing. High near 40F. Winds WSW at 5 to 10 mph." data-track-string="ls_24_hour_ls_24_hour_toggle" className="precip"><div><span class="icon icon-font iconset-weather-data icon-drop

We can see that even we split the html by "\n", each line we still get many html tags in it.<br/>
So, we need to split in each line more than one time to get the data we want.<br/>
We look for lines with temperature.

In [2]:
conditions = []
n = 0
for line in lines:
    if '<td class="temp" headers="hi-lo"' in line:
        n += 1
        # the first one we get is a today data, which we do not need it, so we pass it.
        if n == 1:
            continue
        print(line)

</icon><div class="twc-table-shadow sticky" className="twc-table-shadow sticky"></div></div><span class="day-detail clearfix">DEC 5</span></div></td><td class="description" headers="description" title="Intervals of clouds and sunshine. High 43F. Winds WNW at 10 to 20 mph." data-track-string="ls_24_hour_ls_24_hour_toggle" className="description"><span>Partly Cloudy</span></td><td class="temp" headers="hi-lo" title="Intervals of clouds and sunshine. High 43F. Winds WNW at 10 to 20 mph." data-track-string="ls_24_hour_ls_24_hour_toggle" className="temp"><div><span class="">43<sup>°</sup></span><span class="slash" className="slash"></span><span class="">27<sup>°</sup></span></div></td><td class="precip" headers="precip" title="Intervals of clouds and sunshine. High 43F. Winds WNW at 10 to 20 mph." data-track-string="ls_24_hour_ls_24_hour_toggle" className="precip"><div><span class="icon icon-font iconset-weather-data icon-drop-1" className="icon icon-font iconset-weather-data icon-drop-1"><

The result looks messy. We need to check if the line has only data of one day.<br/>
So we need to find the day of each line.

In [3]:
n = 0
for line in lines:
    if '<td class="temp" headers="hi-lo"' in line:
        n += 1
        # the first one we get is a today data, which we do not need it, so we pass it.
        if n == 1:
            continue
        # this line will spilt the first day it see in the line.
        # if the line has more than one day, it will print only the first one
        print(line.split('<span class="day-detail')[1].split("</span>")[0].split('>')[1])

DEC 5
DEC 8
DEC 12
DEC 13
DEC 17
DEC 18


Some days are missing from the result, and it means some lines contain more than one day of data.<br/>
We need to split by "</tr>" and check the length of the split.

In [4]:
n = 0
for line in lines:
    if '<td class="temp" headers="hi-lo"' in line:
        n += 1
        # the first one we get is a today data, which we do not need it, so we pass it.
        if n == 1:
            continue
        # if split line by tr and get more lenght 2 means there is more that one day in a line
        if len(line.split("</tr>")) == 2:
            print(line.split('<span class="day-detail')[1].split("</span>")[0].split('>')[1])
        else :
            # if the line have more than one day, we split it with tr and find the day in each tr
            tr = line.split("</tr>")
            # for each tr in a line
            for i in range(len(tr)):
                # if not the last item find the day, the last item always not contain day
                if i < len(tr) - 1:
                    print(tr[i].split('<span class="day-detail')[1].split("</span>")[0].split('>')[1])

DEC 5
DEC 6
DEC 7
DEC 8
DEC 9
DEC 10
DEC 11
DEC 12
DEC 13
DEC 14
DEC 15
DEC 16
DEC 17
DEC 18


Now that we got all the dates. Let find the temperature.

In [5]:
n = 0
for line in lines:
    if '<td class="temp" headers="hi-lo"' in line:
        n += 1
        # the first one we get is a today data, which we do not need it, so we pass it.
        if n == 1:
            continue
        # if split line by tr and get more lenght 2 means there is more that one day in a line
        # if there is only one day in the line we use normal split
        if len(line.split("</tr>")) == 2:
            print('day:', line.split('<span class="day-detail')[1].split("</span>")[0].split('>')[1])
            print('high: ', line.split('<td class="temp" headers="hi-lo"')[1].split("</sup>")[0].split('<span class="">')[1].split('<sup>')[0])
            print('low: ', line.split('<td class="temp" headers="hi-lo"')[1].split("</sup>")[1].split('<span class="">')[1].split('<sup>')[0])
        else :
            # if the line have more than one day, we split it with tr and find the day in each tr
            tr = line.split("</tr>")
            # for each tr in a line
            for i in range(len(tr)):
                # if not the last item find the day, the last item always not contain day
                if i < len(tr) - 1:
                    print('day:', tr[i].split('<span class="day-detail')[1].split("</span>")[0].split('>')[1])
                    td = tr[i].split("hi-lo")
                    print('high: ', td[1].split("</sup>")[0].split('<span class="">')[1].split('<sup>')[0])
                    print('low: ', td[1].split("</sup>")[1].split('<span class="">')[1].split('<sup>')[0])

day: DEC 5
high:  43
low:  27
day: DEC 6
high:  38
low:  24
day: DEC 7
high:  30
low:  16
day: DEC 8
high:  39
low:  37
day: DEC 9
high:  53
low:  52
day: DEC 10
high:  59
low:  41
day: DEC 11
high:  43
low:  23
day: DEC 12
high:  32
low:  26
day: DEC 13
high:  40
low:  34
day: DEC 14
high:  44
low:  40
day: DEC 15
high:  45
low:  36
day: DEC 16
high:  40
low:  30
day: DEC 17
high:  41
low:  30
day: DEC 18
high:  36
low:  27


After we get all forecast weather as we need, we calculate the average temperature using a high and low temperature.<br/>
Then we collect the result into a list of string for prediction and send to front-end side.

This is the final code for the application

In [6]:
import requests

r = requests.get('https://weather.com/weather/tenday/l/Boston+MA?canonicalCityId=6320cadd3d539b434b5a45c094becf3edbe8ea88958185a2287a801115c9ae30')
lines = r.text.split('\n')

conditions = []
n = 0
for line in lines:
    if '<td class="temp" headers="hi-lo"' in line:
        n += 1
        if n == 1:
            continue
        condition = {}
        if len(line.split("</tr>")) == 2:
            condition['day'] = line.split('<span class="day-detail')[1].split("</span>")[0].split('>')[1]
            hi = line.split('<td class="temp" headers="hi-lo"')[1].split("</sup>")[0].split('<span class="">')[1].split('<sup>')[0]
            low = line.split('<td class="temp" headers="hi-lo"')[1].split("</sup>")[1].split('<span class="">')[1].split('<sup>')[0]
            condition['temp'] = (float(hi)+float(low))/2
            conditions.append(condition)
        else :
            tr = line.split("</tr>")
            for i in range(len(tr)):
                condition = {}
                if i < len(tr) - 1:
                    condition['day'] = tr[i].split('<span class="day-detail')[1].split("</span>")[0].split('>')[1]
                    td = tr[i].split("hi-lo")
                    hi = td[1].split("</sup>")[0].split('<span class="">')[1].split('<sup>')[0]
                    low = td[1].split("</sup>")[1].split('<span class="">')[1].split('<sup>')[0]
                    condition['temp'] = (float(hi)+float(low))/2
                    conditions.append(condition)
print(conditions)

[{'day': 'DEC 5', 'temp': 35.0}, {'day': 'DEC 6', 'temp': 31.0}, {'day': 'DEC 7', 'temp': 23.0}, {'day': 'DEC 8', 'temp': 38.0}, {'day': 'DEC 9', 'temp': 52.5}, {'day': 'DEC 10', 'temp': 50.0}, {'day': 'DEC 11', 'temp': 33.0}, {'day': 'DEC 12', 'temp': 29.0}, {'day': 'DEC 13', 'temp': 37.0}, {'day': 'DEC 14', 'temp': 42.0}, {'day': 'DEC 15', 'temp': 40.5}, {'day': 'DEC 16', 'temp': 35.0}, {'day': 'DEC 17', 'temp': 35.5}, {'day': 'DEC 18', 'temp': 31.5}]
